In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
## Training set
train = pd.read_csv("/kaggle/input/titanic/train.csv") 

## Test set
test  = pd.read_csv("/kaggle/input/titanic/test.csv") 

## Exploratory Data Analysis

In [ ]:
train.describe()

In [ ]:
train.head().T

In [ ]:
test.head().T

### NULL Check

In [ ]:
train.isnull().sum()

In [ ]:
train.size

In [ ]:
male_age = train[train['Sex'] == 'male'].Age.mean()

In [ ]:
female_age = train[train['Sex'] == 'female'].Age.mean()

In [ ]:
train.loc[train['Sex'] == 'male', 'Age'] = train[train['Sex'] == 'male']['Age'].fillna(male_age)

In [ ]:
train.loc[train['Sex'] == 'female', 'Age'] = train[train['Sex'] == 'female']['Age'].fillna(female_age)

In [ ]:
test.loc[test['Sex'] == 'female', 'Age'] = test[test['Sex'] == 'female']['Age'].fillna(female_age)

In [ ]:
test.loc[test['Sex'] == 'male', 'Age'] = test[test['Sex'] == 'male']['Age'].fillna(male_age)

### Drop Cabin Column

In [ ]:
train.drop(['Cabin'], axis=1, inplace=True)

In [ ]:
test.drop(['Cabin'], axis=1, inplace=True)

### Create family group

In [ ]:
train["family_group"] = train["SibSp"] + train["Parch"] + 1

In [ ]:
test["family_group"] = test["SibSp"] + test["Parch"] + 1

### Drop PassengerId, Ticket, Name, SibSp, Parch

In [ ]:
train.drop(['PassengerId', 'Ticket', 'Name', 'SibSp', 'Parch'], axis=1, inplace=True)
test.drop(['PassengerId', 'Ticket', 'Name', 'SibSp', 'Parch'], axis=1, inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train['Embarked'].unique()

In [ ]:
sns.countplot(train['Embarked'])

In [ ]:
sns.countplot(test['Embarked'])

In [ ]:
train['Embarked'] = train['Embarked'].fillna('S')

In [ ]:
train.isnull().sum()

In [ ]:
sns.lineplot(x=test.Age,y=test.Fare)

In [ ]:
test[test.Fare.isnull()]

In [ ]:
fare_50s = test[(test.Age >= 50) & (test.Age < 60)]['Fare'].mean()

In [ ]:
test['Fare'] = test['Fare'].fillna(fare_50s)

In [ ]:
test.isnull().sum()

### One-Hot Encoding

In [ ]:
dummy_train = pd.get_dummies(train[['Sex', 'Embarked']])
dummy_test  = pd.get_dummies(test[['Sex', 'Embarked']])

train = pd.concat([train.drop(["Sex", "Embarked"], axis = 1), dummy_train], axis = 1)
test  = pd.concat([test.drop(["Sex", "Embarked"], axis = 1), dummy_test], axis = 1)

In [ ]:
train.drop('Sex_female', axis=1, inplace=True)

In [ ]:
test.drop('Sex_female', axis=1, inplace=True)

In [ ]:
train = train.rename(columns={'Sex_male': 'Sex'})

In [ ]:
train

In [ ]:
test = test.rename(columns={'Sex_male': 'Sex'})

In [ ]:
test

In [ ]:
test.isnull().sum()

### Gender investigation

In [ ]:
train.groupby(['Sex', 'Survived']).size().reset_index(name='No. of Survivors')

In [ ]:
## Training Data
sns.countplot(x='Sex', data=train)

In [ ]:
sns.countplot(x='Sex', data=test)

In [ ]:
features = [
                'Pclass',
                'Age',
                'Fare',
                'family_group',
                'Sex',
                'Embarked_C',
                'Embarked_Q',
                'Embarked_S'
            ]
label    = ['Survived']
X_train = train[features]
y_train = train[label]

In [ ]:
X_test = test[features]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,test_size=0.3, random_state=3)

# Modeling

In [ ]:
logisticRegression = LogisticRegression(max_iter=300, solver='liblinear')
logisticRegression.fit(X_train, y_train.values.ravel())

In [ ]:
predictions = logisticRegression.predict(X_valid)

In [ ]:
accuracy = accuracy_score(predictions, y_valid)
print(accuracy * 100)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_valid, predictions))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_valid, predictions)

## Submit

In [ ]:
# One-hot encoding
# test[['Female', 'Male']] = pd.get_dummies(test['Sex'])
testPrediction = logisticRegression.predict(X_test)

In [ ]:
X_test

In [ ]:
## Test set
passenger_id  = pd.read_csv("/kaggle/input/titanic/test.csv")['PassengerId']

In [ ]:
submission = pd.DataFrame({
    "PassengerId": passenger_id,
    "Survived": testPrediction
})

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)